In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yahooquery as yq

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


In [ ]:
emp = pd.read_csv('D:\what\ibovpred\IBOVDia_10-07-23.csv', sep=';', encoding='latin-1')
emp = emp['Código'] + '.SA'
emp = emp.iloc[0:86]

In [ ]:
# Fetch historical data for each stock
stock_data = []
stock_dataAux = []

stock_dataAux = pd.DataFrame(stock_dataAux)

for symbol in emp:
    stock_data = yq.Ticker(symbol).history(period='2y', interval='1d')['close']
    stock_data = pd.DataFrame(stock_data)
    stock_data.reset_index(inplace=True)
    stock_dataAux = pd.concat([stock_dataAux, stock_data], axis=1)
    
stock_dataAux = stock_dataAux['close']
stock_dataAux.columns = emp
stock_dataAux

In [ ]:
#IBOV data
ibov = yq.Ticker('^BVSP').history(period='2y', interval='1d')['close']
ibov = pd.DataFrame(ibov)
ibov.reset_index(inplace=True)
data = pd.concat([ibov, stock_dataAux], axis=1)

data = data.iloc[:, 1:87]
#Renaming the close column to ibov

data.rename(columns={'close': 'ibov'}, inplace=True)

data.shape

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
data_treinamento = data.iloc[:, 2:86].values

normalizador = MinMaxScaler(feature_range=(0,1))
data_treinamento_normalizada = normalizador.fit_transform(data_treinamento)

normalizador_previsao = MinMaxScaler(feature_range=(0,1))
normalizador_previsao.fit_transform(data_treinamento[:,1:2])

In [ ]:
previsores = []
preco_real = []

for i in range(15, 316):
    previsores.append(data_treinamento_normalizada[i-15:i, :])
    preco_real.append(data_treinamento_normalizada[i, 0])
previsores, preco_real = np.array(previsores), np.array(preco_real)

# Validate and reshape the input shape of previsores
num_samples, timesteps, num_features = previsores.shape
previsores_reshaped = previsores.reshape(num_samples, timesteps, num_features)



In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores.shape[1], 84)))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3))

regressor.add(Dense(units = 1, activation = 'sigmoid'))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mean_absolute_error'])

es = EarlyStopping(monitor='loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, verbose=1)
mcp = ModelCheckpoint(filepath='pesos.h5', monitor='loss', save_best_only=True, verbose=1)

regressor.fit(previsores, preco_real, epochs = 100, batch_size = 32, callbacks=[es, rlr, mcp])

regressor.save_weights('regressor_ibov.h5')
